In [5]:
from SAC.Normal_Policy import Normal_Policy
from SAC.Value_function import Value_function
from SAC.SoftActorCritc import SoftActorCritic, plot

import time
import gym
import numpy as np

import sys

sys.path.insert(0, "./../../laser-hockey-env")
import laser_hockey_env as lh
from importlib import reload
import os

Choose your environment you want to visualize

In [6]:
env_name = "Laser_hockey"
env = env = lh.LaserHockeyEnv()
# env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_DEFENSE)
# env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_SHOOTING)

In [7]:
ac_space = env.action_space
o_space = env.observation_space
print(ac_space.low)
print(o_space)

[-1. -1. -1. -1. -1. -1.]
Box(16,)


In [8]:
direct = os.getcwd() + "/SAC/weights/"
version = "v2_full_14"
save_path_new = direct + version

In [9]:
q_fct_config = {"hidden_layers": [256, 256]}
v_fct_config = {"hidden_layers": [256, 256]}
pi_fct_config = {"hidden_layers": [256, 256], "dim": 3, "output_act_fct_mu": None}

SAC_agent = SoftActorCritic(
    o_space=o_space,
    a_space=ac_space,
    env=env,
    value_fct=Value_function,
    policy_fct=Normal_Policy,
    dim_act=3,
    dim_obs=16,
    q_fct_config=q_fct_config,
    v_fct_config=v_fct_config,
    pi_fct_config=pi_fct_config,
    save_path=save_path_new,
)

INFO:tensorflow:Restoring parameters from C:\Users\Maximilian\Documents\Studium\Informatik\3. Semester\Intelligent Systems\project\DeepRLHockey\code/SAC/weights/v2_full_14
restored


In [10]:
player2 = lh.BasicOpponent()

In [11]:
def eval_agent(n=100):
    win = 0
    draw = 0
    lose = 0
    for i in range(n):
        ob = env.reset()

        for _ in range(500):
            env.render()
            obs_agent2 = env.obs_agent_two()
            a1 = SAC_agent.act_greedy(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))[0]
            a2 = player2.act(obs_agent2)

            ob, r, d, info = env.step(np.hstack([a1, a2]))
            if d:
                win += info["winner"] == 1
                lose += info["winner"] == -1
                break
        draw += info["winner"] == 0

    env.close()
    return win / n, draw / n, lose / n

In [15]:
eval_agent(1)

(1.0, 0.0, 0.0)